In [1]:
# Disable tensorflow warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# 0 = all messages are logged (default behavior)
# 1 = INFO messages are not printed
# 2 = INFO and WARNING messages are not printed
# 3 = INFO, WARNING, and ERROR messages are not printed

In [2]:
# Load the TensorBoard notebook extension.
%load_ext tensorboard

In [3]:
from mukkeBude.model import MukkeBudeTransformer
from mukkeBude.mapping import MusicMapping
import mukkeBude.utils as utils
import music21 as m21
import tensorflow as tf
import numpy as np
import keras
from pathlib import Path

# Check if GPU is found
print(tf.config.list_physical_devices('GPU'))

2023-04-20 19:01:17.805508: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


[]


In [4]:
# Create mappings
mapping = MusicMapping.create()
mapping.save("mapping.txt")

In [5]:
# Load songs
paths = m21.corpus.getComposer('bach')
# paths = paths[:100]
print(f"Found {len(paths)} songs in corpus.")

encoded_songs = []
for model_path in paths:
    song = utils.read_single_from_corpus(model_path)
    encoded_song = utils.to_polyphonic_encoding(song, mapping)
    encoded_songs.append(mapping.textify(encoded_song))

print(f"Songs encoded: {len(encoded_songs)}")

Found 100 songs in corpus.


/home/deglasfl/repos/mukkeBude/mukkeBude/utils.py:299: StreamIteratorInefficientWarning: highestTime is not defined on StreamIterators. Call .stream() first for efficiency
  song.flat.getElementsByClass("Note").highestTime,
/home/deglasfl/repos/mukkeBude/mukkeBude/utils.py:300: StreamIteratorInefficientWarning: highestTime is not defined on StreamIterators. Call .stream() first for efficiency
  song.flat.getElementsByClass("Chord").highestTime,
/home/deglasfl/repos/mukkeBude/mukkeBude/utils.py:299: StreamIteratorInefficientWarning: highestTime is not defined on StreamIterators. Call .stream() first for efficiency
  song.flat.getElementsByClass("Note").highestTime,
/home/deglasfl/repos/mukkeBude/mukkeBude/utils.py:300: StreamIteratorInefficientWarning: highestTime is not defined on StreamIterators. Call .stream() first for efficiency
  song.flat.getElementsByClass("Chord").highestTime,
/home/deglasfl/repos/mukkeBude/mukkeBude/utils.py:299: StreamIteratorInefficientWarning: highestTime i

Songs encoded: 100


In [6]:
# Create dataset
utils.create_train_data(encoded_songs, "raw_train_ds.txt")
print("Dataset created")

Dataset created


In [7]:
# Train model
model = MukkeBudeTransformer(mapping)
print(model)

logdir = "logs/bach_transformer"
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

model.train("raw_train_ds.txt", min_training_seq_len=32, epochs=30, tensorboard_callback=tensorboard_callback)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 token_and_position_embeddin  (None, None, 256)        780288    
 g (TokenAndPositionEmbeddin                                     
 g)                                                              
                                                                 
 transformer_decoder (Transf  (None, None, 256)        394749    
 ormerDecoder)                                                   
                                                                 
 transformer_decoder_1 (Tran  (None, None, 256)        394749    
 sformerDecoder)                                                 
                                                                 
 transformer_decoder_2 (Tran  (None, None, 256)        394749

In [8]:
%tensorboard --logdir logs/bach_transformer

In [9]:
# Save the model
model.save("Bach_soloMelodie_transformer")

'/home/deglasfl/repos/mukkeBude/mukkeBude/model/preTrainedModels/Bach_soloMelodie_transformer.h5'

In [ ]:
# Create song
generated_song = model.generate("xxbos n67 d4 n62 d4 n58 d4 n43 d4 xxsep d4 n67 d4 n62 d4 n58 d4 n55 d4 xxsep d4 n69 d4 n62 d4 n57 d4 n54 d4 xxsep", max_length=2048, probability=0.8)
new_song_str = generated_song.numpy().decode("utf-8")
print(f"CLEAN SONG: {new_song_str}\n\n")

print(new_song_str[:50])

In [ ]:
# Convert to music21
new_song_ints = mapping.numericalize(new_song_str.split(" "))
new_song_ints = np.array(new_song_ints)

print(new_song_ints[:50])
print(mapping.textify(new_song_ints[:50]))

new_song = utils.from_polyphonic_encoding(new_song_ints, mapping, bpm=100)
model_path = Path("generated_song_bach.mid")
utils.write_midi(new_song, model_path)
utils.write_musicxml(new_song, "generated_song_bach.musicxml")

In [ ]:
# Open song in MuseScore (Linux if installed)
!musescore4portable {path.absolute()}